In [1]:
import pandas as pd 
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

: 

In [6]:
df=pd.read_csv('F:\\Shipping_Pricing_Prediction\\Notebook\Preprocessed_df.csv')

In [7]:
df.head()

,PO_/_SO_#,ASN/DN_#,Country,Fulfill_Via,Vendor_INCO_Term,Shipment_Mode,Sub_Classification,Unit_of_Measure_(Per_Pack),Line_Item_Quantity,Line_Item_Value,Pack_Price,Unit_Price,First_Line_Designation,Weight_(Kilograms),Freight_Cost_(USD),Line_Item_Insurance_(USD),Days_to_Process
0,SCMS,ASN,Côte d'Ivoire,Direct Drop,EXW,Air,HIV test,30,19,551.0,29.00,0.97,Yes,13.0,780.34,NaN,-947
1,SCMS,ASN,Vietnam,Direct Drop,EXW,Air,Pediatric,240,1000,6200.0,6.20,0.03,Yes,358.0,4521.50,NaN,-782
2,SCMS,ASN,Côte d'Ivoire,Direct Drop,FCA,Air,HIV test,100,500,40000.0,80.00,0.80,Yes,171.0,1653.78,NaN,-861
3,SCMS,ASN,Vietnam,Direct Drop,EXW,Air,Adult,60,31920,127360.8,3.99,0.07,Yes,1855.0,16007.06,NaN,-856
4,SCMS,ASN,Vietnam,Direct Drop,EXW,Air,Adult,60,38000,121600.0,3.20,0.05,Yes,7590.0,45450.08,NaN,-877


In [10]:
df.shape

(10324, 17)

In [11]:
df.duplicated().sum()

61

In [12]:
# Drop duplicates 
df.drop_duplicates(inplace=True)

# Feature Transformation and Scalling 

In [13]:
import seaborn as sns
import numpy as np

In [14]:
cat_features=[cols for cols in df.columns if df[cols].dtype=='O']
cat_features

['PO_/_SO_#',
 'ASN/DN_#',
 'Country',
 'Fulfill_Via',
 'Vendor_INCO_Term',
 'Shipment_Mode',
 'Sub_Classification',
 'First_Line_Designation']

In [15]:
num_features=['Unit_of_Measure_(Per_Pack)',
 'Line_Item_Quantity',
 'Pack_Price',
 'Unit_Price',
 'Weight_(Kilograms)',
 'Freight_Cost_(USD)',
 'Line_Item_Insurance_(USD)',
 'Days_to_Process']


# Experiment no 1 :use of SimpleImputer and Robustscaler for numeric data and onehot encoder for categorical and Simpleimputer for missing data

In [8]:
x=df.drop('Line_Item_Value',axis=1)
y=df['Line_Item_Value']

In [17]:
x.head()

,PO_/_SO_#,ASN/DN_#,Country,Fulfill_Via,Vendor_INCO_Term,Shipment_Mode,Sub_Classification,Unit_of_Measure_(Per_Pack),Line_Item_Quantity,Pack_Price,Unit_Price,First_Line_Designation,Weight_(Kilograms),Freight_Cost_(USD),Line_Item_Insurance_(USD),Days_to_Process
0,SCMS,ASN,Côte d'Ivoire,Direct Drop,EXW,Air,HIV test,30,19,29.00,0.97,Yes,13.0,780.34,NaN,-947
1,SCMS,ASN,Vietnam,Direct Drop,EXW,Air,Pediatric,240,1000,6.20,0.03,Yes,358.0,4521.50,NaN,-782
2,SCMS,ASN,Côte d'Ivoire,Direct Drop,FCA,Air,HIV test,100,500,80.00,0.80,Yes,171.0,1653.78,NaN,-861
3,SCMS,ASN,Vietnam,Direct Drop,EXW,Air,Adult,60,31920,3.99,0.07,Yes,1855.0,16007.06,NaN,-856
4,SCMS,ASN,Vietnam,Direct Drop,EXW,Air,Adult,60,38000,3.20,0.05,Yes,7590.0,45450.08,NaN,-877


In [18]:
x.isna().sum().sort_values(ascending=False)

Freight_Cost_(USD)            2438
Weight_(Kilograms)            1482
Shipment_Mode                  358
Line_Item_Insurance_(USD)      285
PO_/_SO_#                        0
ASN/DN_#                         0
Country                          0
Fulfill_Via                      0
Vendor_INCO_Term                 0
Sub_Classification               0
Unit_of_Measure_(Per_Pack)       0
Line_Item_Quantity               0
Pack_Price                       0
Unit_Price                       0
First_Line_Designation           0
Days_to_Process                  0
dtype: int64

In [13]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10263 entries, 0 to 10323
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PO_/_SO_#                   10263 non-null  object 
 1   ASN/DN_#                    10263 non-null  object 
 2   Country                     10263 non-null  object 
 3   Fulfill_Via                 10263 non-null  object 
 4   Vendor_INCO_Term            10263 non-null  object 
 5   Shipment_Mode               9905 non-null   object 
 6   Sub_Classification          10263 non-null  object 
 7   Unit_of_Measure_(Per_Pack)  10263 non-null  int64  
 8   Line_Item_Quantity          10263 non-null  int64  
 9   Pack_Price                  10263 non-null  float64
 10  Unit_Price                  10263 non-null  float64
 11  First_Line_Designation      10263 non-null  object 
 12  Weight_(Kilograms)          8781 non-null   float64
 13  Freight_Cost_(USD)          7825 non

In [19]:
scaler=RobustScaler()

In [20]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())])

In [21]:
# Create transformers for categorical and numerical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('onehot', OneHotEncoder(drop='first',handle_unknown='ignore'))
])

In [22]:
# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, cat_features),
        ('num', numerical_transformer, num_features)
    ])

In [23]:
# Create the final pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

In [24]:
# Fit and transform the data
X_transformed = pipeline.fit_transform(x)

In [25]:
X_transformed

<10263x67 sparse matrix of type '<class 'numpy.float64'>'
	with 134615 stored elements in Compressed Sparse Row format>

In [26]:
df['Line_Item_Value'].describe()

count    1.026300e+04
mean     1.580605e+05
std      3.460482e+05
min      0.000000e+00
25%      4.392585e+03
50%      3.052350e+04
75%      1.664648e+05
max      5.951990e+06
Name: Line_Item_Value, dtype: float64

In [27]:
# Reshape the target variable to fit the scaler's input format
y = y.values.reshape(-1, 1)

In [28]:
# Fit and transform the target variable
y_transformed = scaler.fit_transform(y)

In [29]:
# Reshape back to a 1D array
y_transformed = y_transformed.flatten()

In [30]:
# Split the data into train and test sets
X_train, X_test, y_train_scaled, y_test_scaled = train_test_split(X_transformed, y_transformed,test_size=0.2, random_state=42)

In [31]:
print(X_train.shape,X_test.shape)

(8210, 67) (2053, 67)


In [32]:
print(y_train_scaled.shape,y_test_scaled.shape)

(8210,) (2053,)


In [34]:
# Dictionary which contains models for experiment
models_list = {
    "Random Forest": RandomForestRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
     "K-Neighbors Regressor": KNeighborsRegressor(),
    "XGBClassifier": XGBRegressor(), 
    "AdaBoost Regressor": AdaBoostRegressor(),
    'LinearRegression':LinearRegression(),
    'LassoCV':LassoCV()
}


In [33]:
def evaluate_regression(true, predicted):
    '''
    This function takes in true values and predicted values
    Returns: RMSE, MSE, MAE, R-squared, Adjusted R-squared
    '''
    mse = mean_squared_error(true, predicted)  # Calculate MSE
    r2 = r2_score(true, predicted)  # Calculate R-squared

    # Calculate Adjusted R-squared
    n = len(true)
    p = 1  # Number of features used for prediction (change if you have more features)
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

 
    return  mse, r2, adj_r2


In [31]:
# Create a function which can evaluate models and return a report 
def evaluate_models(X, y, models):
    '''
    This function takes in X and y and models dictionary as input
    It splits the data into Train Test split
    Iterates through the given model dictionary and evaluates the metrics
    Returns: Dataframe which contains report of all models metrics with cost
    '''
    models_list = []
    train_mse=[]
    test_mse=[]
    train_r2_score=[]
    test_r2_score=[]
    train_adjusted_r2_score=[]
    test_adjusted_r2_score=[]
    # separate dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
    
    # Create cost of the model as per data description
    

    for i in range(len(list(models))):
        model = list(models.values())[i]
        model.fit(X_train, y_train) # Train model

        # Make predictions
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        # Training set performance
        model_train_mse,model_train_r2,model_train_adjusted_r2=evaluate_regression(y_train,y_train_pred)
        train_mse.append(model_train_mse)
        train_r2_score.append(model_train_r2)
        train_adjusted_r2_score.append(model_train_adjusted_r2)

        # Test set performance
        model_test_mse,model_test_r2,model_test_adjusted_r2=evaluate_regression(y_test,y_test_pred)
        test_mse.append(model_test_mse)
        test_r2_score.append(model_test_r2)
        test_adjusted_r2_score.append(model_test_adjusted_r2)

        print(list(models.keys())[i])
        models_list.append(list(models.keys())[i])

    
        print('Model performance for Training set')
        print('- MSE: {:.4f}'.format(model_train_mse)) 
        print('- R2_score: {:.4f}'.format(model_train_r2))
        print('- Adjusted R2 Score: {:.4f}'.format(model_train_adjusted_r2))

        print('----------------------------------')

        print('Model performance for Test set')
        print('- MSE: {:.4f}'.format(model_test_mse)) 
        print('- R2_score: {:.4f}'.format(model_test_r2))
        print('- Adjusted R2 Score: {:.4f}'.format(model_test_adjusted_r2))
    
        print('='*35)
        print('\n')
        
    report = pd.DataFrame(list(zip(models_list, train_mse, train_r2_score, train_adjusted_r2_score, test_mse, test_r2_score, test_adjusted_r2_score)), columns=['Model Name', 'Train_MSE', 'Train_R2', 'Train_Adjusted_r2', 'Test_MSE', 'Test_R2', 'Test_Adjusted_R2']).sort_values(by=["Test_Adjusted_R2"], ascending=False)
    return report

In [32]:
report1=evaluate_models(X_transformed,y_transformed,models_list)
report1

Random Forest
Model performance for Training set
- MSE: 0.0109
- R2_score: 0.9977
- Adjusted R2 Score: 0.9977
----------------------------------
Model performance for Test set
- MSE: 0.0509
- R2_score: 0.9875
- Adjusted R2 Score: 0.9875


Decision Tree
Model performance for Training set
- MSE: 0.0000
- R2_score: 1.0000
- Adjusted R2 Score: 1.0000
----------------------------------
Model performance for Test set
- MSE: 0.1486
- R2_score: 0.9636
- Adjusted R2 Score: 0.9636


Gradient Boosting
Model performance for Training set
- MSE: 0.0435
- R2_score: 0.9907
- Adjusted R2 Score: 0.9907
----------------------------------
Model performance for Test set
- MSE: 0.0701
- R2_score: 0.9828
- Adjusted R2 Score: 0.9828


K-Neighbors Regressor
Model performance for Training set
- MSE: 0.1686
- R2_score: 0.9640
- Adjusted R2 Score: 0.9639
----------------------------------
Model performance for Test set
- MSE: 0.1763
- R2_score: 0.9568
- Adjusted R2 Score: 0.9568


XGBClassifier
Model performance 

,Model Name,Train_MSE,Train_R2,Train_Adjusted_r2,Test_MSE,Test_R2,Test_Adjusted_R2
0,Random Forest,1.088700e-02,0.997673,0.997672,0.050945,0.987513,0.987507
4,XGBClassifier,8.313064e-04,0.999822,0.999822,0.057252,0.985967,0.985960
2,Gradient Boosting,4.353714e-02,0.990693,0.990692,0.070135,0.982810,0.982801
1,Decision Tree,3.240946e-34,1.000000,1.000000,0.148594,0.963579,0.963561
3,K-Neighbors Regressor,1.686343e-01,0.963951,0.963947,0.176269,0.956795,0.956774
6,LinearRegression,2.736146e-01,0.941510,0.941503,0.274379,0.932748,0.932715
7,LassoCV,2.787223e-01,0.940418,0.940411,0.283891,0.930417,0.930383
5,AdaBoost Regressor,1.209750e+00,0.741394,0.741362,1.260916,0.690941,0.690791


In 1st experiment RandomForest Regressor is performing well without hypertunning let do crossvalidation

In [33]:
# Split the data into train and test sets
X_train, X_test, y_train_scaled, y_test_scaled = train_test_split(X_transformed, y_transformed,test_size=0.2, random_state=42)

In [34]:
random=RandomForestRegressor()
xgb=XGBRegressor()
gd=GradientBoostingRegressor()

In [35]:
model1=random.fit(X_train,y_train_scaled)
model2=xgb.fit(X_train,y_train_scaled)
model3=gd.fit(X_train,y_train_scaled)

In [36]:
from sklearn import model_selection
from sklearn import metrics

In [37]:
cv = model_selection.KFold(n_splits=8)

In [38]:
for (train, test), i in zip(cv.split(X_train, y_train_scaled), range(8)):
    y_pred1 = model1.predict(X_test)
    y_pred2 = model2.predict(X_test)
    y_pred3 = model3.predict(X_test)
    print (metrics.mean_squared_error(y_test_scaled, y_pred1))
    print (metrics.mean_squared_error(y_test_scaled, y_pred2))
    print (metrics.mean_squared_error(y_test_scaled, y_pred3))
    print("*******************************************************")
    

0.050157234570880053
0.057252116868023076
0.07058141504611018
*******************************************************
0.050157234570880053
0.057252116868023076
0.07058141504611018
*******************************************************
0.050157234570880053
0.057252116868023076
0.07058141504611018
*******************************************************
0.050157234570880053
0.057252116868023076
0.07058141504611018
*******************************************************
0.050157234570880053
0.057252116868023076
0.07058141504611018
*******************************************************
0.050157234570880053
0.057252116868023076
0.07058141504611018
*******************************************************
0.050157234570880053
0.057252116868023076
0.07058141504611018
*******************************************************
0.050157234570880053
0.057252116868023076
0.07058141504611018
*******************************************************


In [39]:
# Define the models you want to evaluate
models = {
    "Random Forest": RandomForestRegressor(),
    "XGB": XGBRegressor(),
    "Gradient Boosting": GradientBoostingRegressor()
}

# Define the scoring metrics as individual strings
scoring = {
    'mse': make_scorer(mean_squared_error, greater_is_better=False),
    'r2': 'r2',
    'mae': make_scorer(mean_absolute_error, greater_is_better=False)
}

# Loop through each model and perform cross-validation
for model_name, model in models.items():
    print(f"Model: {model_name}")
    
    # Separate dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split(X_transformed, y_transformed,test_size=0.2, random_state=42)
    
    # Fit the model
    model.fit(X_train, y_train)
  
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculate scores using cross-validation
    for metric_name, metric_func in scoring.items():
        print(f"{metric_name.capitalize()}:")
        
        # Perform cross-validation and calculate the mean score
        scores = cross_val_score(model, X_train, y_train, cv=5, scoring=metric_func)
        mean_score = scores.mean()
        print("  Mean:", mean_score)        
        print("="*35)
    print("\n")

Model: Random Forest
Mse:
  Mean: -0.08536714810081684
R2:
  Mean: 0.9813948561686872
Mae:
  Mean: -0.10142037759264515


Model: XGB
Mse:
  Mean: -0.06494415389896299
R2:
  Mean: 0.9863222708322539
Mae:
  Mean: -0.07837734203560152


Model: Gradient Boosting
Mse:
  Mean: -0.09167382324518371
R2:
  Mean: 0.9800075711477317
Mae:
  Mean: -0.13319755307872785




After cross validation XGB Regressor is performing well 

Exeperiment NO2 : Simple Imputer by median + Robust Scaler-->Numeric data ,missing_value by constant ,one hot encoding -->categorical data

* Why not min_max scaler or StandardScaler ?
Since our data is not normaley distributed and numeric data has so many outlier robust scaler will treat outlier 



In [40]:
x=df.drop('Line_Item_Value',axis=1)
y=df['Line_Item_Value']

In [41]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())])

In [42]:
# Create transformers for categorical and numerical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(drop='first',handle_unknown='ignore'))
])

In [43]:
# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, cat_features),
        ('num', numerical_transformer, num_features)
    ])

In [44]:
# Create the final pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

In [45]:
scaler=RobustScaler()

In [46]:
# Reshape the target variable to fit the scaler's input format
y = y.values.reshape(-1, 1)

In [47]:
# Fit and transform the target variable
y_transformed = scaler.fit_transform(y)

In [48]:
# Reshape back to a 1D array
y_transformed = y_transformed.flatten()

In [49]:
# Fit and transform the data
X_transformed_2 = pipeline.fit_transform(x)

In [50]:
# Dictionary which contains models for experiment
models_list = {
    "Random Forest": RandomForestRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
     "K-Neighbors Regressor": KNeighborsRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "AdaBoost Regressor": AdaBoostRegressor(),
    'LinearRegression':LinearRegression(),
    'LassoCV':LassoCV()
}


In [51]:
report2=evaluate_models(X_transformed_2,y_transformed,models_list)
report2

Random Forest
Model performance for Training set
- MSE: 0.0110
- R2_score: 0.9977
- Adjusted R2 Score: 0.9977
----------------------------------
Model performance for Test set
- MSE: 0.0518
- R2_score: 0.9873
- Adjusted R2 Score: 0.9873


Decision Tree
Model performance for Training set
- MSE: 0.0000
- R2_score: 1.0000
- Adjusted R2 Score: 1.0000
----------------------------------
Model performance for Test set
- MSE: 0.1555
- R2_score: 0.9619
- Adjusted R2 Score: 0.9619


Gradient Boosting
Model performance for Training set
- MSE: 0.0435
- R2_score: 0.9907
- Adjusted R2 Score: 0.9907
----------------------------------
Model performance for Test set
- MSE: 0.0706
- R2_score: 0.9827
- Adjusted R2 Score: 0.9827


K-Neighbors Regressor
Model performance for Training set
- MSE: 0.1690
- R2_score: 0.9639
- Adjusted R2 Score: 0.9639
----------------------------------
Model performance for Test set
- MSE: 0.1761
- R2_score: 0.9568
- Adjusted R2 Score: 0.9568


XGBRegressor
Model performance f

,Model Name,Train_MSE,Train_R2,Train_Adjusted_r2,Test_MSE,Test_R2,Test_Adjusted_R2
0,Random Forest,1.097248e-02,0.997654,0.997654,0.051842,0.987293,0.987287
4,XGBRegressor,8.944796e-04,0.999809,0.999809,0.057150,0.985992,0.985985
2,Gradient Boosting,4.353714e-02,0.990693,0.990692,0.070581,0.982700,0.982692
1,Decision Tree,3.240946e-34,1.000000,1.000000,0.155493,0.961888,0.961869
3,K-Neighbors Regressor,1.689510e-01,0.963884,0.963879,0.176146,0.956826,0.956804
6,LinearRegression,2.729632e-01,0.941649,0.941642,0.272586,0.933187,0.933155
7,LassoCV,2.787223e-01,0.940418,0.940411,0.283891,0.930417,0.930383
5,AdaBoost Regressor,1.589517e+00,0.660211,0.660170,1.641028,0.597773,0.597577


Lets do cross validation for above models 

In [52]:
# Define the models you want to evaluate
models = {
    "Random Forest": RandomForestRegressor(),
    "XGB": XGBRegressor(),
    "Gradient Boosting": GradientBoostingRegressor()
}

# Define the scoring metrics as individual strings
scoring = {
    'mse': make_scorer(mean_squared_error, greater_is_better=False),
    'r2': 'r2',
    'mae': make_scorer(mean_absolute_error, greater_is_better=False)
}

# Loop through each model and perform cross-validation
for model_name, model in models.items():
    print(f"Model: {model_name}")
    
    # Separate dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split(X_transformed_2, y_transformed,test_size=0.2, random_state=42)
    
    # Fit the model
    model.fit(X_train, y_train)
  
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculate scores using cross-validation
    for metric_name, metric_func in scoring.items():
        print(f"{metric_name.capitalize()}:")
        
        # Perform cross-validation and calculate the mean score
        scores = cross_val_score(model, X_train, y_train, cv=5, scoring=metric_func)
        mean_score = scores.mean()
        print("  Mean:", mean_score)        
        print("="*35)
    print("\n")

Model: Random Forest
Mse:
  Mean: -0.0888492916635406
R2:
  Mean: 0.9812742678784385
Mae:
  Mean: -0.10177665186459242


Model: XGB
Mse:
  Mean: -0.06525044853809378
R2:
  Mean: 0.9862415725035799
Mae:
  Mean: -0.07859298072569401


Model: Gradient Boosting
Mse:
  Mean: -0.09148292804675252
R2:
  Mean: 0.9799634380064507
Mae:
  Mean: -0.13318302752098546




# Final Model

* After cross validation XGBRegressor is performing well

In [35]:
x.head()

,PO_/_SO_#,ASN/DN_#,Country,Fulfill_Via,Vendor_INCO_Term,Shipment_Mode,Sub_Classification,Unit_of_Measure_(Per_Pack),Line_Item_Quantity,Pack_Price,Unit_Price,First_Line_Designation,Weight_(Kilograms),Freight_Cost_(USD),Line_Item_Insurance_(USD),Days_to_Process
0,SCMS,ASN,Côte d'Ivoire,Direct Drop,EXW,Air,HIV test,30,19,29.00,0.97,Yes,13.0,780.34,NaN,-947
1,SCMS,ASN,Vietnam,Direct Drop,EXW,Air,Pediatric,240,1000,6.20,0.03,Yes,358.0,4521.50,NaN,-782
2,SCMS,ASN,Côte d'Ivoire,Direct Drop,FCA,Air,HIV test,100,500,80.00,0.80,Yes,171.0,1653.78,NaN,-861
3,SCMS,ASN,Vietnam,Direct Drop,EXW,Air,Adult,60,31920,3.99,0.07,Yes,1855.0,16007.06,NaN,-856
4,SCMS,ASN,Vietnam,Direct Drop,EXW,Air,Adult,60,38000,3.20,0.05,Yes,7590.0,45450.08,NaN,-877


In [36]:
y=df['Line_Item_Value']

In [37]:
y = y.values.reshape(-1, 1)

In [38]:
y_transformed=scaler.fit_transform(y)

In [39]:
y_transformed=y_transformed.flatten()

In [44]:
y_final=y_transformed

In [45]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['PO_/_SO_#', 'ASN/DN_#',
                                                   'Country', 'Fulfill_Via',
                                                   'Vendor_INCO_Term',
                                                   'Shipment_Mode',
                                                   'Sub_Classification',
                                                   'First_Line_Designation']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['Unit_of_Measure_(Per_Pack)',
                                                   'Line_Item_Quantity',
                                                   'Pack_Price', 'Unit_Price',
                                                   'Weight_(Kilograms)',
                                                   'Freight_Cost_(USD)',
                                                   'Line_Item_Insurance_(USD)',
                                                   'Days_to_Process'])]))])

In [46]:
X_final=pipeline.fit_transform(x)

In [47]:
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(X_final, y_final,test_size=0.2, random_state=42)

In [48]:
final_model=XGBRegressor()
final_model.fit(X_train_final,y_train_final)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [49]:
# Make predictions
y_train_pred = final_model.predict(X_train)
y_test_pred = final_model.predict(X_test)

In [84]:
# Assuming you have a Robust Scaler object named 'target_scaler'
original_predictions = y_test_pred.reshape(-1)
original_predictions = scaler.inverse_transform(original_predictions.reshape(-1, 1))


In [90]:
first_prediction_numeric = original_predictions[3][0]
first_prediction_numeric

174302.33

In [50]:
train_mse,train_r2,train_adjusted_r2=evaluate_regression(y_train_final,y_train_pred)
test_mse,test_r2,test_adjusted_r2=evaluate_regression(y_test_final,y_test_pred)

print("train-mse-->",train_mse)
print("train-r2-->",train_r2)
print("train-adjusted-r2",train_adjusted_r2)
print("*************************************************")
print("test-mse-->",test_mse)
print("test-r2-->",test_r2)
print("test-adjusted-r2",test_adjusted_r2)

train-mse--> 0.000831306428900952
train-r2--> 0.9998222928825992
train-adjusted-r2 0.9998222712321219
*************************************************
test-mse--> 0.057252116868023076
test-r2--> 0.9859671324433662
test-adjusted-r2 0.9859602904796624


# Saving Final Model and Predicting for new data

In [132]:
y=df['Line_Item_Value']
x=df.drop('Line_Item_Value',axis=1)

In [ ]:
# Create transformers for categorical and numerical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('onehot', OneHotEncoder(drop='first',handle_unknown='ignore'))
])
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())])

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, cat_features),
        ('num', numerical_transformer, num_features)
    ])

# Create the final pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])
scaler=RobustScaler()
# Reshape the target variable to fit the scaler's input format
y = y.values.reshape(-1, 1)

# Fit and transform the target variable
y_transformed = scaler.fit_transform(y)

y_transformed = y_transformed.flatten()


In [26]:

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
import joblib

# Assuming you have your X and y prepared already

y=df['Line_Item_Value']
x=df.drop('Line_Item_Value',axis=1)

target_scaler=RobustScaler()
# Reshape the target variable to fit the scaler's input format
y = y.values.reshape(-1, 1)

# Fit and transform the target variable
y_transformed = target_scaler.fit_transform(y)

y_transformed = y_transformed.flatten()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y_transformed, test_size=0.2, random_state=42)

# Create transformers for categorical and numerical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))
])
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

# Define categorical and numerical features (replace with your actual feature lists)
cat_features =['PO_/_SO_#','ASN/DN_#','Country','Fulfill_Via','Vendor_INCO_Term','Shipment_Mode','Sub_Classification','First_Line_Designation'] # List of categorical feature column names
num_features = ['Unit_of_Measure_(Per_Pack)','Line_Item_Quantity','Pack_Price','Unit_Price','Weight_(Kilograms)','Freight_Cost_(USD)','Line_Item_Insurance_(USD)','Days_to_Process'] # List of numerical feature column names

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, cat_features),
        ('num', numerical_transformer, num_features)
    ])

# Create an XGBoostRegressor
xgb_model = XGBRegressor(n_estimators=100, max_depth=3)  # Modify parameters as needed

# Create a pipeline that includes preprocessing and XGBoost model
final_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', TransformedTargetRegressor(regressor=xgb_model, transformer=target_scaler))
])

# Fit the final pipeline on the training data
final_pipeline.fit(X_train, y_train)

# Evaluate the model
score = final_pipeline.score(X_test, y_test)
print(f"Model R-squared score: {score}")

# Save the final pipeline and model
joblib.dump(final_pipeline, 'final_pipeline.pkl')




Model R-squared score: 0.9894892200037565


['final_pipeline.pkl']

In [27]:
# To predict on new incoming data:
import joblib
loaded_pipeline = joblib.load('final_pipeline.pkl')

In [28]:
loaded_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['PO_/_SO_#', 'ASN/DN_#',
                                                   'Country', 'Fulfill_Via',
                                                   'Vendor_INCO_Term',
                                                   'Shipment_Mode',
                                                   'Sub_Classification',
                                                   'First_Line_Designati...
                                                                   importance_type=None,
                                                                   interaction_constraints=None,
                                                                   learning_rate=None,
                                                                   max_bin=None,
                                                                   max_cat_threshold=None,
                                                                   max_cat_to_onehot=None,
                                                                   max_delta_step=None,
                                                                   max_depth=3,
                                                                   max_leaves=None,
                                                                   min_child_weight=None,
                                                                   missing=nan,
                                                                   monotone_constraints=None,
                                                                   n_estimators=100,
                                                                   n_jobs=None,
                                                                   num_parallel_tree=None,
                                                                   predictor=None,
                                                                   random_state=None, ...),
                                            transformer=RobustScaler()))])

In [29]:
data=pd.DataFrame([x.iloc[3700]],index=[0])

In [30]:
data

,PO_/_SO_#,ASN/DN_#,Country,Fulfill_Via,Vendor_INCO_Term,Shipment_Mode,Sub_Classification,Unit_of_Measure_(Per_Pack),Line_Item_Quantity,Pack_Price,Unit_Price,First_Line_Designation,Weight_(Kilograms),Freight_Cost_(USD),Line_Item_Insurance_(USD),Days_to_Process
0,SCMS,ASN,Vietnam,Direct Drop,EXW,Air,Adult,60,18406,11.0,0.18,Yes,2571.0,13880.57,334.07,191


In [31]:
preprocessed_new_data = loaded_pipeline.named_steps['preprocessor'].transform(data)

In [32]:
preprocessed_new_data

<1x66 sparse matrix of type '<class 'numpy.float64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [33]:
# Make predictions on the preprocessed new data
new_data_predictions = loaded_pipeline.named_steps['model'].predict(preprocessed_new_data)

In [34]:
new_data_predictions

array([1.1310856], dtype=float32)

In [35]:
new_data_predictions[0]

1.1310856

In [38]:
scaled_prediction=loaded_pipeline.named_steps['model'].predict(preprocessed_new_data)
scaled_prediction

array([1.1310856], dtype=float32)

In [47]:
# Inverse transform the scaled prediction using the original scaler
original_scale_prediction = target_scaler.inverse_transform(scaled_prediction.reshape(-1, 1))

print("Predicted target value for new data (original scale):", original_scale_prediction[0][0])

Predicted target value for new data (original scale): 213857.27


The actual value for Line_Item_Value is 202466 we are close to our predictions

In [57]:
data2=pd.DataFrame([df.iloc[9000]],index=[0])

In [58]:
data2

,PO_/_SO_#,ASN/DN_#,Country,Fulfill_Via,Vendor_INCO_Term,Shipment_Mode,Sub_Classification,Unit_of_Measure_(Per_Pack),Line_Item_Quantity,Line_Item_Value,Pack_Price,Unit_Price,First_Line_Designation,Weight_(Kilograms),Freight_Cost_(USD),Line_Item_Insurance_(USD),Days_to_Process
0,SO,DN,Zimbabwe,From RDC,N/A - From RDC,Truck,Adult,60,9073,22773.23,2.51,0.04,Yes,521.0,2494.56,49.1,239


In [59]:
preprocessed_new_data2 = loaded_pipeline.named_steps['preprocessor'].transform(data2)

In [60]:
# Make predictions on the preprocessed new data
new_data_predictions2 = loaded_pipeline.named_steps['model'].predict(preprocessed_new_data2)

In [61]:
scaled_prediction2=loaded_pipeline.named_steps['model'].predict(preprocessed_new_data2)
scaled_prediction2
# Inverse transform the scaled prediction using the original scaler
original_scale_prediction2 = target_scaler.inverse_transform(scaled_prediction2.reshape(-1, 1))

print("Predicted target value for new data (original scale):", original_scale_prediction2[0][0])

Predicted target value for new data (original scale): 21953.521


In [56]:
df.iloc[9000]

PO_/_SO_#                                 SO
ASN/DN_#                                  DN
Country                             Zimbabwe
Fulfill_Via                         From RDC
Vendor_INCO_Term              N/A - From RDC
Shipment_Mode                          Truck
Sub_Classification                     Adult
Unit_of_Measure_(Per_Pack)                60
Line_Item_Quantity                      9073
Line_Item_Value                     22773.23
Pack_Price                              2.51
Unit_Price                              0.04
First_Line_Designation                   Yes
Weight_(Kilograms)                     521.0
Freight_Cost_(USD)                   2494.56
Line_Item_Insurance_(USD)               49.1
Days_to_Process                          239
Name: 9000, dtype: object